In [1]:
from pyspark import *

In [2]:
#Step 1: Read the table from the spark data
census = spark.read.table("adult_csv")
#Step2 : Reading the Table Schema
census.printSchema()

root
-- age: integer (nullable = true)
-- employer_type: string (nullable = true)
-- numf1: integer (nullable = true)
-- education: string (nullable = true)
-- numf2: integer (nullable = true)
-- marital: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- gender: string (nullable = true)
-- capitalgain: integer (nullable = true)
-- capitalloss: integer (nullable = true)
-- hr_per_wk: integer (nullable = true)
-- region: string (nullable = true)
-- income: string (nullable = true)

In [3]:
#census1 = census
import pyspark.sql.functions as F
step1 = [F.when(~F.col(x).isin("?","NULL", "NA", "NaN"), F.col(x)).alias(x)  for x in census.columns] 
step2 = census.select(*step1).dropna(how='any')
step3 = step2.drop('numf1','numf2') 
#numf2 is dropped because its numerical lable for education column so its pointless to have both so i used education
#numf1 is finalweight and was beyound the comprehension as to how the datat collectors arrived at that(kaggale)

In [4]:
# normalize my numeric data

from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# UDF for converting column type from vector to double type
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())


# VectorAssembler Transformation - Converting column to vector type
assembler = [VectorAssembler(inputCols= [col], outputCol="Vect_"+col) for col in ['capitalgain','capitalloss']]
# MinMaxScaler Transformation
scaler = [MinMaxScaler(inputCol="Vect_"+col, outputCol="Scaled_"+col) for col in ['capitalgain','capitalloss'] ]

# Pipeline of VectorAssembler and MinMaxScaler
all_stages = assembler + scaler 
pipeline = Pipeline(stages= all_stages)

# Fitting pipeline on dataframe
step4 = pipeline.fit(step3).transform(step3).drop('capitalgain','capitalloss','Vect_capitalgain','Vect_capitalloss').withColumn('Scaled_capitalgain', unlist('Scaled_capitalgain')).withColumn('Scaled_capitalloss', unlist('Scaled_capitalloss'))

In [5]:
from pyspark.ml.feature import Bucketizer
from pyspark.ml import Pipeline
#for age

bucketizer_age = Bucketizer(splits=[0.0, 30.0, 45.0, 60.0,float("inf")], inputCol="age", outputCol="index_age_group")
bucketizer_hrwk = Bucketizer(splits=[0.0, 25.0, 45.0, float("inf")], inputCol="hr_per_wk", outputCol="index_hr/wk")
bucketizer_cg = Bucketizer(splits=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, float("inf")], inputCol="Scaled_capitalgain", outputCol="index_capital_gain")
bucketizer_cl = Bucketizer(splits=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, float("inf")], inputCol="Scaled_capitalloss", outputCol="index_capital_loss")
# Transform original data into its bucket index.
step5 = bucketizer_age.transform(step4).drop("age")
step5 = bucketizer_hrwk.transform(step5).drop("hr_per_wk")
step5 = bucketizer_cg.transform(step5).drop("Scaled_capitalgain")
step5 = bucketizer_cl.transform(step5).drop("Scaled_capitalloss")
step5 = step5.withColumnRenamed("index_age_group","age").withColumnRenamed("index_hr/wk","hr/wk").withColumnRenamed("index_capital_gain","capital_gain").withColumnRenamed("index_capital_loss","capital_loss")

In [6]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# Clearly identifying the job categories
def employementtype(job):
  if(job=='Never-worked'or job=='Without-pay'):
    return "Unemployed"
  if(job=='State-gov' or job=='Local-gov'):
    return "Govt"
  if(job=='Self-emp-inc' or job=='Self-emp-not-inc'):
    return "self_emp"
  else:
    return(job)
  
# For Distinguishing mrital status into 3 categories    
def maritalstat(mar):
  #mar= as.character(mar)
  
  if mar in ['Separated','Widowed','Divorced']:
    return "Not-Married"
  elif mar== 'Never-married':
    return "Never-married"
  else: 
    return "Married"

  
# For Distinguishing countries into continents  
Asia=['China','Hong','India','Iran','Cambodia','Japan', 'Laos',
        ' Philippines' ,' Vietnam' ,' Taiwan', ' Thailand']
North_America = ['Canada','United-States','Puerto-Rico' ]

Europe = ['England' ,'France', 'Germany' ,'Greece','Holand-Netherlands','Hungary',
            'Ireland','Italy','Poland','Portugal','Scotland','Yugoslavia']

Latin_and_SouthAmerica = ['Columbia','Cuba','Dominican-Republic','Ecuador',
                             'El-Salvador','Guatemala',' Haiti',' Honduras',
                             'Mexico','Nicaragua','Outlying-US(Guam-USVI-etc)','Peru',
                             'Jamaica','Trinadad&Tobago']
Other= ['South']

def regionlist(countries):
  if countries in Asia:
    return "Asia"
  if countries in North_America:
    return "North America"
  if countries in Europe:
    return "Europe"
  if countries in Latin_and_SouthAmerica:
    return "Latin & South America"
  else:
    return "Others"  


#CALLING USER DEFINED FUNCTIONS
etype_udf = udf(employementtype,StringType())
step6 = step5.withColumn("employer_type", etype_udf("employer_type"))

# udf(user defined function) the function and its type and then use withcolumn to apply the transformation
marital_udf = udf(maritalstat,StringType())
step6 = step6.withColumn("marital", marital_udf("marital"))

region_udf = udf(regionlist,StringType())
step6 = step6.withColumn("region",region_udf("region"))

In [8]:
#CHECK FOR DISTINCT VALUES in every feature/variable/column
for col,dtype in step6.dtypes:
    print(col)
    step6.select(col).distinct().show()
    print("----------------------------------------")

employer_type
+-------------+
employer_type|
+-------------+
 Govt|
 self_emp|
 Private|
 Federal-gov|
 Unemployed|
+-------------+

----------------------------------------
education
+------------+
 education|
+------------+
 10th|
 Masters|
 5th-6th|
 Assoc-acdm|
 Assoc-voc|
 7th-8th|
 9th|
 HS-grad|
 Bachelors|
 11th|
 1st-4th|
 Preschool|
 12th|
 Doctorate|
Some-college|
 Prof-school|
+------------+

----------------------------------------
marital
+-------------+
 marital|
+-------------+
 Married|
Never-married|
 Not-Married|
+-------------+

----------------------------------------
occupation
+-----------------+
 occupation|
+-----------------+
 Sales|
 Exec-managerial|
 Prof-specialty|
Handlers-cleaners|
 Farming-fishing|
 Craft-repair|
 Transport-moving|
 Priv-house-serv|
 Protective-serv|
 Other-service|
 Tech-support|
Machine-op-inspct|
 Armed-Forces|
 Adm-clerical|
+-----------------+

----------------------------------------
relationship
+--------------+
 relationship|
+--------------+
 Own-child|
 Not-in-family|
 Unmarried|
 Wife|
Other-relative|
 Husband|
+--------------+

----------------------------------------
race
+------------------+
 race|
+------------------+
 Other|
Amer-Indian-Eskimo|
 White|
Asian-Pac-Islander|
 Black|
+------------------+

----------------------------------------
gender
+------+
gender|
+------+
Female|
 Male|
+------+

----------------------------------------
region
+--------------------+
 region|
+--------------------+
 Europe|
Latin & South Ame...|
 North America|
 Others|
 Asia|
+--------------------+

----------------------------------------
income
+------+
income|
+------+
 <=50K|
 >50K|
+------+

----------------------------------------
age
+---+
age|
+---+
0.0|
1.0|
3.0|
2.0|
+---+

----------------------------------------
hr/wk
+-----+
hr/wk|
+-----+
 0.0|
 1.0|
 2.0|
+-----+

----------------------------------------
capital_gain
+------------+
capital_gain|
+------------+
 0.0|
 1.0|
 4.0|
 3.0|
 2.0|
 9.0|
+------------+

----------------------------------------
capital_loss
+------------+
capital_loss|
+------------+
 8.0|
 0.0|
 7.0|
 1.0|
 4.0|
 3.0|
 2.0|
 6.0|
 5.0|
 9.0|
+------------+

----------------------------------------

In [9]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

#feature_column = [col for col,dtype  in modeldata.dtypes if dtype =='string']

#STRING INDEXER FOR ALL THE COLUMNS WITH DATATYPES== STRING
String_index = [StringIndexer(inputCol=col, outputCol='index_'+col) for col,dtype  in step6.dtypes if dtype =='string' if col!='income']
String_index += [StringIndexer(inputCol='income', outputCol='label')]


#ONEHOT INDEXER FOR ALL THE COLUMNS NUMERICAL TYPE
Onehot_index = [OneHotEncoder(dropLast=False, inputCol='index_'+col, outputCol='encoded_'+col) for col,dtype  in step6.dtypes if col not in ['age', 'hr/wk','capital_gain','capital_loss','income' ] ]
Onehot_index += [OneHotEncoder(inputCol='age', outputCol='encoded_age')]
Onehot_index += [OneHotEncoder(inputCol='hr/wk', outputCol='encoded_hr/wk')]
Onehot_index += [OneHotEncoder(inputCol='capital_gain', outputCol='encoded_capital_gain')]
Onehot_index += [OneHotEncoder(inputCol='capital_loss', outputCol='encoded_capital_loss')]


#GENERATING ONE FEATURE VECTOR WHICH WILL GO INTO MACHINE LEARNING MODEL
vectorassembler_stage = VectorAssembler(inputCols=['encoded_' + col for col in step6.columns if col!='income'], outputCol='features')


#PRINT ALL THE JOBS
print(String_index)
print(Onehot_index)
print(vectorassembler_stage)

[StringIndexer_a22b7d49f3d2, StringIndexer_f2773918c272, StringIndexer_0404aa3b5559, StringIndexer_2066bb819a9e, StringIndexer_a5550fc5e278, StringIndexer_eb1f9a2a70e0, StringIndexer_fc160bfdd89b, StringIndexer_3a42d3025194, StringIndexer_6c765fa3c89d]
[OneHotEncoder_b53ea4f3cc7c, OneHotEncoder_3be82cb7d518, OneHotEncoder_3b0a68973136, OneHotEncoder_f8e5fc279d47, OneHotEncoder_ae7d93d1846a, OneHotEncoder_09e22c9674a5, OneHotEncoder_71e78229f2d1, OneHotEncoder_1cbc30d21d1d, OneHotEncoder_7f59f63a5a56, OneHotEncoder_111019691761, OneHotEncoder_fe834d93033d, OneHotEncoder_640b89833f6c]
VectorAssembler_b1372761dc71

In [10]:
#PIPELINE CALLED ALL STAGE ==>> STRING INDEXING + ONE HOT ENCODING + VECTOR ASSEMBLER
all_stages = String_index + Onehot_index + [vectorassembler_stage]
pipelined_fit = Pipeline(stages=all_stages).fit(step6)

# SELECTING ONLY THE ENCODED COLUMNS TO SEE THE TRANSFORMATION
final_columns = ['encoded_'+col for col,dtype  in step6.dtypes if col!='income'  ] + ['features', 'label']
step7 = pipelined_fit.transform(step6).select(final_columns)
display(step7.limit(10))

encoded_employer_type,encoded_education,encoded_marital,encoded_occupation,encoded_relationship,encoded_race,encoded_gender,encoded_region,encoded_age,encoded_hr/wk,encoded_capital_gain,encoded_capital_loss,features,label
"List(0, 5, List(0), List(1.0))","List(0, 16, List(5), List(1.0))","List(0, 3, List(1), List(1.0))","List(0, 14, List(6), List(1.0))","List(0, 6, List(2), List(1.0))","List(0, 5, List(1), List(1.0))","List(0, 2, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 3, List(0), List(1.0))","List(0, 2, List(1), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 79, List(0, 10, 22, 30, 40, 45, 49, 51, 56, 60, 61, 70), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",0.0
"List(0, 5, List(0), List(1.0))","List(0, 16, List(0), List(1.0))","List(0, 3, List(0), List(1.0))","List(0, 14, List(9), List(1.0))","List(0, 6, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 2, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 3, List(1), List(1.0))","List(0, 2, List(), List())","List(0, 9, List(0), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 79, List(0, 5, 21, 33, 38, 44, 49, 51, 57, 61, 70), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",0.0
"List(0, 5, List(2), List(1.0))","List(0, 16, List(6), List(1.0))","List(0, 3, List(0), List(1.0))","List(0, 14, List(11), List(1.0))","List(0, 6, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 2, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 3, List(0), List(1.0))","List(0, 2, List(1), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 79, List(2, 11, 21, 35, 38, 44, 49, 51, 56, 60, 61, 70), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",1.0
"List(0, 5, List(0), List(1.0))","List(0, 16, List(1), List(1.0))","List(0, 3, List(0), List(1.0))","List(0, 14, List(6), List(1.0))","List(0, 6, List(0), List(1.0))","List(0, 5, List(1), List(1.0))","List(0, 2, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 3, List(1), List(1.0))","List(0, 2, List(1), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 79, List(0, 6, 21, 30, 38, 45, 49, 51, 57, 60, 61, 70), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",1.0
"List(0, 5, List(0), List(1.0))","List(0, 16, List(7), List(1.0))","List(0, 3, List(1), List(1.0))","List(0, 14, List(5), List(1.0))","List(0, 6, List(1), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 2, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 3, List(1), List(1.0))","List(0, 2, List(1), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 79, List(0, 12, 22, 29, 39, 44, 49, 51, 57, 60, 61, 70), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",0.0
"List(0, 5, List(1), List(1.0))","List(0, 16, List(9), List(1.0))","List(0, 3, List(0), List(1.0))","List(0, 14, List(1), List(1.0))","List(0, 6, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 2, List(0), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 3, List(), List())","List(0, 2, List(1), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 79, List(1, 14, 21, 25, 38, 44, 49, 51, 60, 61, 70), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",1.0
"List(0, 5, List(0), List(1.0))","List(0, 16, List(1), List(1.0))","List(0, 3, List(1), List(1.0))","List(0, 14, List(5), List(1.0))","List(0, 6, List(3), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 2, List(1), List(1.0))","List(0, 5, List(0), List(1.0))","List(0, 3, List(0), List(1.0))","List(0, 2, List(1), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 9, List(0), List(1.0))","List(0, 79, List(0, 6, 22, 29, 41, 44, 50, 51, 56, 60, 61, 70), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",0.0
"List(0, 5, List(0), List(1.0))","List(0, 16, List(8), List(1

# MACHINE LEARNING Models

In [12]:
#SELECTING COLUMNS FOR MACHINE LEARNING MODEL
mldata = step7.select("features","label")

#1: Logistic Regression

In [14]:
(trainDF, testDF) = mldata.randomSplit([0.8, 0.2], seed=1231)
trainDF.cache()
testDF.cache()

Out[16]: DataFrame[features: vector, label: double]

In [15]:
from pyspark.ml.classification import LogisticRegression

# Load training data


lr = LogisticRegression()

# Fit the model
lrModel = lr.fit(trainDF)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))
lrModel.summary.accuracy

Coefficients: [0.786306134702478,0.5536625875241842,0.6094202585570349,1.2007010221944803,-0.52741408154112,0.03794175323260771,0.4062885644429996,1.1266285088671333,1.4434934821212024,0.4446186426812252,-0.6711358868919073,0.5733166630492471,-0.6650406145024941,-0.9716258901631141,1.9785612400779131,-1.1304810218293004,-0.14726173705590412,1.9309636273461876,-1.1375344171382036,-1.0874732611509743,-2.3591255202186465,1.1790566376416283,-0.10662998712512779,0.22405112653370024,0.2969213049434663,0.8110573321188834,1.0939174966735685,0.34612029068213374,0.5731466430964612,-0.5778892407224351,-0.021084321548565883,0.17769719004912624,-0.39718071682471606,-0.6210714785582705,0.9129775769169968,0.814442490487104,-0.9251725387108303,0.0429181325518015,1.0734740566388,0.20932501872306825,-0.6087418565936467,-0.03432353975382116,2.0777149383407294,-0.2530416563595561,1.3133682823733925,1.0848888310881546,1.599036320465884,0.6122206792611494,1.2099747888402486,1.0063602434933978,0.3911940319189905,2.2758136904234503,1.6053079055655584,2.481275066402028,1.7169116761301695,1.7473139592996512,-0.975174548928968,0.0606099642309338,0.4486005512315896,-1.511983123102963,-0.45762051021013483,-10.805044329316067,-4.55899142649114,9.362937223215114,-19.524804244358407,-21.035214578002318,0.0,0.0,0.0,0.0,3.0438199539795296,2.6739400846472514,-11.416549418794306,2.388665772576111,4.610025498912995,4.912979875276808,7.7323834514450285,-3.599874302357182,-8.16666342732313]
Intercept: -0.8050893414640454
Out[17]: 0.845414774615129

In [16]:
#for test df
result = lrModel.transform(testDF)
result.select("prediction","label","features").show(10)

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
+----------+-----+--------------------+
only showing top 10 rows

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
print("AUC: %(result)s" % {"result": evaluator.evaluate(result)})
display(lrModel, trainDF, "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.9997586412226628
0.0,0.047619047619047616,0.9997586412226628
0.0,0.09523809523809523,0.9964595733598345
0.0,0.14285714285714285,0.8410533217230212
0.0,0.19047619047619047,0.8343621720435789
0.012345679012345678,0.19047619047619047,0.812373482632714
0.012345679012345678,0.23809523809523808,0.7940022908330541
0.012345679012345678,0.2857142857142857,0.7868378812055389
0.012345679012345678,0.3333333333333333,0.7533533727829665
0.012345679012345678,0.42857142857142855,0.7439042883617669


In [18]:
display(lrModel, trainDF)

fitted values,residuals
-2.0298666724904644,-0.1161026038095926
-1.5722461622803294,-0.17189641925823632
-0.9940821593305625,-0.2701065291953623
5.639961330495028,0.003540426640165517
-0.47994613215514365,-0.3822648454185685
-0.717856821177567,0.6721349000111311
-0.3298662341769112,-0.41827317079940224
0.12775427603322276,-0.5318952002616767
0.12775427603322276,0.4681047997383233
-1.3726977500535278,-0.20218433398150668


In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
evaluator = BinaryClassificationEvaluator(labelCol='label')

# We have only two choices: area under ROC and PR curves :-(
auroc = evaluator.evaluate(result, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(result, {evaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))

Area under ROC Curve: 0.9060
Area under PR Curve: 0.7665

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
lr.setRegParam(0.08)

model = lr.fit(trainDF)
#training = model.transform(trainDF)
result = model.transform(testDF)
print("evaluations %(result)s" % {"result": BinaryClassificationEvaluator().evaluate(result)})

evaluations 0.9035111044157303

In [21]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType

#important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels = result.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')
#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())


[[6521. 316.]
 [1154. 1115.]]

In [22]:
# We are trying for all possible evaluations possible

In [23]:
from pyspark.ml.classification import LogisticRegression
logr = LogisticRegression(featuresCol='features', labelCol='label')


In [24]:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
      addGrid(logr.regParam, [0, 0.1, 0.2, 0.5, 1]).\
      addGrid(logr.elasticNetParam, [0, 0.1, 0.2, 0.5, 1]).\
      addGrid(logr.maxIter, [5,10,20,50,100]).\
      build()

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

In [26]:
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=logr, evaluator=evaluator, estimatorParamMaps=param_grid, numFolds=3)
cv_model = cv.fit(trainDF)  # fitiing data to my cross validation model

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [27]:
show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_cv = cv_model.transform(trainDF)
pred_training_cv.select(show_columns).show(5, truncate=False)

+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
only showing top 5 rows

In [28]:
pred_test_cv = cv_model.transform(testDF)
pred_test_cv.select(show_columns).show(5, truncate=False)

+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[2.0298666724904644,-2.0298666724904644]|[0.8838973961904073,0.1161026038095926]|
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
only showing top 5 rows

In [29]:
print('Intercept: ' + str(cv_model.bestModel.intercept) + "\n" 'coefficients: ' + str(cv_model.bestModel.coefficients))

Intercept: -0.8050893414640454
coefficients: [0.786306134702478,0.5536625875241842,0.6094202585570349,1.2007010221944803,-0.52741408154112,0.03794175323260771,0.4062885644429996,1.1266285088671333,1.4434934821212024,0.4446186426812252,-0.6711358868919073,0.5733166630492471,-0.6650406145024941,-0.9716258901631141,1.9785612400779131,-1.1304810218293004,-0.14726173705590412,1.9309636273461876,-1.1375344171382036,-1.0874732611509743,-2.3591255202186465,1.1790566376416283,-0.10662998712512779,0.22405112653370024,0.2969213049434663,0.8110573321188834,1.0939174966735685,0.34612029068213374,0.5731466430964612,-0.5778892407224351,-0.021084321548565883,0.17769719004912624,-0.39718071682471606,-0.6210714785582705,0.9129775769169968,0.814442490487104,-0.9251725387108303,0.0429181325518015,1.0734740566388,0.20932501872306825,-0.6087418565936467,-0.03432353975382116,2.0777149383407294,-0.2530416563595561,1.3133682823733925,1.0848888310881546,1.599036320465884,0.6122206792611494,1.2099747888402486,1.0063602434933978,0.3911940319189905,2.2758136904234503,1.6053079055655584,2.481275066402028,1.7169116761301695,1.7473139592996512,-0.975174548928968,0.0606099642309338,0.4486005512315896,-1.511983123102963,-0.45762051021013483,-10.805044329316067,-4.55899142649114,9.362937223215114,-19.524804244358407,-21.035214578002318,0.0,0.0,0.0,0.0,3.0438199539795296,2.6739400846472514,-11.416549418794306,2.388665772576111,4.610025498912995,4.912979875276808,7.7323834514450285,-3.599874302357182,-8.16666342732313]

In [30]:
print('Logistic Regression', "\n",'The best RegParam is: ', cv_model.bestModel._java_obj.getRegParam(), "\n",'The best ElasticNetParam is:', cv_model.bestModel._java_obj.getElasticNetParam(), "\n",'The best Iteration is:',cv_model.bestModel._java_obj.getMaxIter() , "\n", 'Area under ROC is:', cv_model.bestModel.summary.areaUnderROC)

Logistic Regression 
 The best RegParam is: 0.0 
 The best ElasticNetParam is: 0.0 
 The best Iteration is: 100 
 Area under ROC is: 0.9018748530240799

In [31]:
cv_model.avgMetrics

Out[50]: [0.8896683662782221,
 0.8998937195983895,
 0.9004334619977938,
 0.9004664877390309,
 0.9005118127822711,
 0.889668366278217,
 0.8998937195983905,
 0.9004334619977958,
 0.9004664877390317,
 0.9005118127822545,
 0.8896683662782174,
 0.8998937195983865,
 0.9004334619977958,
 0.9004664877390306,
 0.9005118127822631,
 0.8896683662782192,
 0.8998937195983838,
 0.9004334619977985,
 0.9004664877390277,
 0.900511812782258,
 0.8896683662782183,
 0.8998937195983867,
 0.9004334619977941,
 0.9004664877390358,
 0.9005118127822707,
 0.8933768234315995,
 0.8971648203240595,
 0.897614202504573,
 0.89760454668057,
 0.8976045466805695,
 0.8804202245080925,
 0.8922807520051561,
 0.8924828005312846,
 0.892577082726433,
 0.8927571779318053,
 0.8756781787718766,
 0.8855767276454898,
 0.8828874541957528,
 0.8838672878717075,
 0.8838793794601774,
 0.8593286192959567,
 0.8612872294797924,
 0.8591056153790473,
 0.8605500343469199,
 0.8614497035616788,
 0.8038190268070557,
 0.7786148503665706,
 0.7757425006851344,
 0.7733662923002796,
 0.7733662923002796,
 0.89209015602976,
 0.8949482192144185,
 0.8955081629206852,
 0.8954914158217552,
 0.895491415821754,
 0.8758315576264503,
 0.8826604374887821,
 0.8832491138529024,
 0.8835890070575485,
 0.8833868855418585,
 0.8659119978788501,
 0.8697065120254273,
 0.8684789499538681,
 0.8701622360061434,
 0.8703633552967125,
 0.8063172258502393,
 0.7864405089275053,
 0.7786148503665706,
 0.7786148503665706,
 0.7786148503665706,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.8891977161478004,
 0.8916334086986604,
 0.891695200419202,
 0.8916968639807932,
 0.8916968639807943,
 0.8591611122309617,
 0.8610635223953156,
 0.8615313157820117,
 0.8625823280143265,
 0.8626500987636727,
 0.7999105762272873,
 0.8018867702854015,
 0.8004309106029105,
 0.8004309106029105,
 0.8004309106029105,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.8873408027460119,
 0.8898560340655057,
 0.8884224181596265,
 0.888422418159627,
 0.8884224181596293,
 0.8004309106029105,
 0.7999105762272875,
 0.7999105762272875,
 0.7999105762272875,
 0.7999105762272875,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5]

#2: Support Vector Machine

In [33]:
from pyspark.ml.classification import LinearSVC

# Load training data


lsvc = LinearSVC(maxIter=10, regParam=0.1)

# Fit the model
lsvcModel = lsvc.fit(trainDF)

# Print the coefficients and intercept for linear SVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

lsvcresult = lsvcModel.transform(testDF)
lsvcresult.select("prediction","label","features").show(10)

#Compute accuracy of test
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print("evaluation: %(result)s" % {"result": evaluator.evaluate(lsvcresult)})

Coefficients: [0.0,-0.033743860875586305,-0.06477774972258915,0.022164113427037852,-0.21179801067761536,-0.08973154033787008,-0.031848486609035116,0.18160775872780818,0.41057161856582985,-0.020728404228870158,-0.11371412081946108,-0.0007163794148134117,-0.13936661024532507,-0.21183712161427687,0.8214301829066173,-0.18376042082601177,-0.10066431671059341,0.7176998228731105,-0.18844295275242354,-0.2015068877965634,-0.16429809707444507,0.2511151567637143,-0.056610560156204186,-0.04462316299942209,-0.048707346508502884,0.16144530315256336,0.18211156078088106,-0.021836216624686284,0.028937469065061516,-0.13243627639627115,-0.10598758156902964,-0.08510144784920891,-0.09982490363582461,-0.21377504173027667,0.09108760494542731,-0.006866516148532331,-0.06506099871283452,0.00420658890194933,0.052301841366817814,-0.046743746808640775,-0.09594057997325951,-0.05034099568580736,0.13807419687463188,-0.09704598123258705,0.02564558935444294,-0.050905492599615146,-0.0459658919972455,-0.12571214711379328,-0.059639479765212486,-6.904500097902799e-05,-0.033895412158631025,0.04561538702473436,-0.13674362251872793,-0.06163674259586478,-0.07935919175859421,-0.02510199498882443,-0.11512791634740474,0.005715799427070747,0.1625257416398855,-0.1316428400068919,-0.059449465963983275,-0.9259984143045604,1.4314481102159486,1.440789220201523,-1.0981018660120525,-1.1701570243335409,0.0,0.0,0.0,0.0,-0.23029300405220185,-0.3343296573856788,-0.671603280514063,-0.5193233994776925,0.6066680991693245,0.2915480126573509,1.2383064015413692,-0.136425999589121,-0.33523425313342764]
Intercept: -0.009131763063426845
+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
+----------+-----+--------------------+
only showing top 10 rows

evaluation: 0.8901680399851699

In [34]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
svmevaluator = BinaryClassificationEvaluator()

# We have only two choices: area under ROC and PR curves :-(
svmauroc = svmevaluator.evaluate(lsvcresult, {svmevaluator.metricName: "areaUnderROC"})
svmauprc = svmevaluator.evaluate(lsvcresult, {svmevaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(svmauroc))
print("Area under PR Curve: {:.4f}".format(svmauprc))

Area under ROC Curve: 0.8902
Area under PR Curve: 0.7328

In [35]:
#ESTIMATOR
from pyspark.ml.classification import LinearSVC
lsvm = LinearSVC(featuresCol='features', labelCol='label')

#GRID VECTOR

from pyspark.ml.tuning import ParamGridBuilder
param_grid_svm = ParamGridBuilder().\
      addGrid(lsvm.regParam, [0, 0.1, 0.2, 0.5, 1]).\
      addGrid(lsvm.maxIter, [5,10,20,50,100]).\
      build()

#Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
svmevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#CROSS VALIDATION
from pyspark.ml.tuning import CrossValidator
cv_svm = CrossValidator(estimator=lsvm, evaluator=svmevaluator, estimatorParamMaps=param_grid_svm, numFolds=3)
cv_svm_model = cv_svm.fit(trainDF)  # fitiing data to my cross validation model

show_columns = ['features', 'label', 'prediction', 'rawPrediction']
pred_training_svm = cv_svm_model.transform(trainDF)
pred_training_svm.select(show_columns).show(5, truncate=False)

pred_test_svm = cv_svm_model.transform(testDF)
pred_test_svm.select(show_columns).show(5, truncate=False)

print('Support Vector Machine', "\n",'The best RegParam is: ', cv_svm_model.bestModel._java_obj.getRegParam(),  "\n",'The best Iteration is:',cv_svm_model.bestModel._java_obj.getMaxIter() , "\n", 'Area under ROC is:', svmevaluator.evaluate(pred_test_svm, {svmevaluator.metricName: "areaUnderROC"}))

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
features |label|prediction|rawPrediction |
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
only showing top 5 rows

+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
features |label|prediction|rawPrediction |
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[1.5502744362605185,-1.5502744362605185]|
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+
only showing top 5 rows

Support Vector Machine 
 The best RegParam is: 0.0 
 The best Iteration is: 100 
 Area under ROC is: 0.9002626996588012

In [36]:
cv_svm_model.avgMetrics

Out[47]: [0.8801328381388824,
 0.8828244764122053,
 0.8947447206456531,
 0.8954773556221034,
 0.8955864451565398,
 0.8784523672933836,
 0.8868713845479568,
 0.8914949066817095,
 0.892747807622652,
 0.8929788992054499,
 0.8777807523155428,
 0.8861316995036613,
 0.8896649122764579,
 0.891711302239333,
 0.8917845460874403,
 0.8796245103946323,
 0.8861596714201143,
 0.8873581874206293,
 0.8884618773905053,
 0.8887571520509476,
 0.8794383172044258,
 0.8831719989651949,
 0.8860843878457356,
 0.8875743412838755,
 0.887648780327474]

#3: Naive Bayes

In [38]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
nbmodel = nb.fit(trainDF)

# select example rows to display.
nbresult = model.transform(testDF)
nbresult.select("prediction","label","features").show(10)

# compute accuracy on the test set
nbevaluator = BinaryClassificationEvaluator()
accuracy = nbevaluator.evaluate(nbresult)
print("evaluations: %(nbresult)s" % {"nbresult": nbevaluator.evaluate(nbresult)})

+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
+----------+-----+--------------------+
only showing top 10 rows

evaluations: 0.9035111044157295

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
nbevaluator = BinaryClassificationEvaluator()

# We have only two choices: area under ROC and PR curves :-(
nbauroc = nbevaluator.evaluate(nbresult, {nbevaluator.metricName: "areaUnderROC"})
nbauprc = nbevaluator.evaluate(nbresult, {nbevaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(nbauroc))
print("Area under PR Curve: {:.4f}".format(nbauprc))

Area under ROC Curve: 0.9035
Area under PR Curve: 0.7582

In [40]:
#ESTIMATOR
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol='features', labelCol='label')

#GRID VECTOR

from pyspark.ml.tuning import ParamGridBuilder
param_grid_nb = ParamGridBuilder().\
      addGrid(nb.smoothing, [0.0,1.0,2.0,4.0,6.0,8.0]).\
      addGrid(nb.modelType, ["multinomial", "bernoulli"]).\
      build()

#Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
nbevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#CROSS VALIDATION
from pyspark.ml.tuning import CrossValidator
cv_nb = CrossValidator(estimator=nb, evaluator=nbevaluator, estimatorParamMaps=param_grid_nb, numFolds=3)
cv_nb_model = cv_nb.fit(trainDF)  # fitiing data to my cross validation model

show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_nb = cv_nb_model.transform(trainDF)
pred_training_nb.select(show_columns).show(5, truncate=False)

pred_test_nb = cv_nb_model.transform(testDF)
pred_test_nb.select(show_columns).show(5, truncate=False)

print('Naive Bayes ',"\n",'The best Smoothening is: ', cv_nb_model.bestModel._java_obj.getSmoothing(), "\n",'The best model type is:', cv_nb_model.bestModel._java_obj.getModelType(), "\n", 'Area under ROC is:', nbevaluator.evaluate(pred_test_nb, {nbevaluator.metricName: "areaUnderROC"}))
cv_nb_model.avgMetrics

##nbauroc = nbevaluator.evaluate(nbresult, {pred_test_nb.metricName: "areaUnderROC"})
#nbauprc = nbevaluator.evaluate(nbresult, {pred_test_nb.metricName: "areaUnderPR"})


MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
only showing top 5 rows

+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[-37.50684453802806,-39.10566176696517]|[0.8318530116420073,0.1681469883579927]|
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
only showing top 5 rows

Naive Bayes 
 The best Smoothening is: 0.0 
 The best model type is: multinomial 
 Area under ROC is: 0.8008644664305177

In [41]:
cv_nb_model.avgMetrics

Out[46]: [0.7967954044240257,
 0.773598523269748,
 0.7967271715887003,
 0.7734518898564935,
 0.7966523254479954,
 0.7733613277827187,
 0.7965258818135919,
 0.7731625370724926,
 0.7963875324863571,
 0.7729787503336294,
 0.7962399754722813,
 0.7727947994406505]

# Random Forest

In [43]:
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Train a RandomForest model.
rf = RandomForestClassifier( numTrees=10)


# Train model.  This also runs the indexers.
rfmodel = rf.fit(trainDF)

# Make predictions.
rfresult = rfmodel.transform(testDF)

# Select example rows to display.
rfresult.select("prediction","label","features").show(10)

# Select (prediction, true label) and compute test error
rfevaluator = BinaryClassificationEvaluator()
print("evaluations: %(rfresult)s" % {"rfresult": rfevaluator.evaluate(rfresult)})



+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
+----------+-----+--------------------+
only showing top 10 rows

evaluations: 0.8702890701845087

In [44]:
# We have only two choices: area under ROC and PR curves :-(
rfauroc = rfevaluator.evaluate(rfresult, {rfevaluator.metricName: "areaUnderROC"})
rfauprc = rfevaluator.evaluate(rfresult, {rfevaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(rfauroc))
print("Area under PR Curve: {:.4f}".format(rfauprc))

Area under ROC Curve: 0.8703
Area under PR Curve: 0.6924

In [45]:
#ESTIMATOR
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Train a RandomForest model.
rf = RandomForestClassifier(featuresCol='features', labelCol='label')

#GRID VECTOR
from pyspark.ml.tuning import ParamGridBuilder
param_grid_rf = ParamGridBuilder().\
      addGrid(rf.impurity,['gini']).\
      addGrid(rf.maxDepth, [2, 3, 4]).\
      addGrid(rf.minInfoGain, [0.0, 0.1, 0.2, 0.3]).\
      addGrid(rf.numTrees,[20,40,60,80,100]).\
      build()

#Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
rfevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#CROSS VALIDATION
from pyspark.ml.tuning import CrossValidator
cv_rf = CrossValidator(estimator=rf, evaluator=rfevaluator, estimatorParamMaps=param_grid_rf, numFolds=3)
cv_rf_model = cv_rf.fit(trainDF)  # fitiing data to my cross validation model

show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_rf = cv_rf_model.transform(trainDF)
pred_training_rf.select(show_columns).show(5, truncate=False)

pred_test_rf = cv_rf_model.transform(testDF)
pred_test_rf.select(show_columns).show(5, truncate=False)

print('Random forest ',"\n",'The best Max Depth is: ', cv_rf_model.bestModel._java_obj.getMaxDepth(), "\n",'The best min Info gain is:', cv_rf_model.bestModel._java_obj.getMinInfoGain(), "\n", 'Area under ROC is:', rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderROC"}))
#rfmodel.trees
##nbauroc = rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderROC"})
#nbauprc = rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderPR"})

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
only showing top 5 rows

+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[43.220331569424246,16.779668430575736]|[0.7203388594904043,0.2796611405095957]|
+------------------------------------------------------------------------------------------+-----+----------+---------------------------------------+---------------------------------------+
only showing top 5 rows

Random forest 
 The best Max Depth is: 4 
 The best min Info gain is: 0.0 
 Area under ROC is: 0.8784316766552892

In [46]:
cv_rf_model.avgMetrics

Out[63]: [0.8467468727815348,
 0.856713290801483,
 0.8643139645545602,
 0.8606172339432743,
 0.8603757437411175,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.8579541318497603,
 0.8607442834777863,
 0.8648662126361153,
 0.8635001870314647,
 0.864399495971802,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.8688613431282997,
 0.8679062517913052,
 0.8730238428321284,
 0.8694100473342277,
 0.8710812800892465,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5]

#5: Gradient Boost

In [48]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Train a GBT model.
gb = GBTClassifier()
# Chain indexers and GBT in a Pipeline

# Train model.  This also runs the indexers.
gbmodel = gb.fit(trainDF)

# Make predictions.
gbresult = gbmodel.transform(testDF)

# Select example rows to display.
gbresult.select("prediction","label","features").show(5)

# Select (prediction, true label) and compute test error
gbevaluator = BinaryClassificationEvaluator()

print("evaluations: %(gbresult)s" % {"gbresult": gbevaluator.evaluate(gbresult)})


+----------+-----+--------------------+
prediction|label| features|
+----------+-----+--------------------+
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
 0.0| 0.0|(79,[0,5,21,24,38...|
+----------+-----+--------------------+
only showing top 5 rows

evaluations: 0.8976359609165223

In [49]:
# We have only two choices: area under ROC and PR curves :-(
gbauroc = gbevaluator.evaluate(gbresult, {gbevaluator.metricName: "areaUnderROC"})
gbauprc = gbevaluator.evaluate(gbresult, {gbevaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(gbauroc))
print("Area under PR Curve: {:.4f}".format(gbauprc))

Area under ROC Curve: 0.8976
Area under PR Curve: 0.7440

In [50]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# ESTIMATOR
gbt = GBTClassifier(featuresCol='features', labelCol='label')


#GRID VECTOR
from pyspark.ml.tuning import ParamGridBuilder
param_grid_gbt = ParamGridBuilder().\
    addGrid(gbt.maxDepth, [2, 3, 4]).\
    addGrid(gbt.minInfoGain, [0.0, 0.1, 0.2]).\
    addGrid(gbt.stepSize, [0.02, 0.05, 0.1]).\
    addGrid(gb.maxIter,[20,40,60,80,100]).\
    build()

#Evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
gbtevaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

#CROSS VALIDATION
from pyspark.ml.tuning import CrossValidator
cv_gbt = CrossValidator(estimator=gbt, evaluator=gbtevaluator, estimatorParamMaps=param_grid_gbt)
cv_gbt_model = cv_gbt.fit(trainDF)  # fitiing data to my cross validation model

show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']
pred_training_gbt = cv_gbt_model.transform(trainDF)
pred_training_gbt.select(show_columns).show(5, truncate=False)

pred_test_gbt = cv_gbt_model.transform(testDF)
pred_test_gbt.select(show_columns).show(5, truncate=False)


print('Gradient Boosting ',"\n",'The best Max Depth is: ', cv_gbt_model.bestModel._java_obj.getMaxDepth(), "\n",'The best min Info gain is:',cv_gbt_model.bestModel._java_obj.getMinInfoGain(), "\n", 'step size: ', cv_gbt_model.bestModel._java_obj.getStepSize(),"\n" ,'Area under ROC is:', gbtevaluator.evaluate(pred_test_gbt, {gbtevaluator.metricName: "areaUnderROC"}))

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
only showing top 5 rows

+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
features |label|prediction|rawPrediction |probability |
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
(79,[0,5,21,24,38,44,49,51,56,60,61,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0.0 |0.0 |[0.8386282111334181,-0.8386282111334181]|[0.842540894325519,0.15745910567448096]|
+------------------------------------------------------------------------------------------+-----+----------+----------------------------------------+---------------------------------------+
only showing top 5 rows

Gradient Boosting 
 The best Max Depth is: 4 
 The best min Info gain is: 0.0 
 step size: 0.1 
 Area under ROC is: 0.8934300783341733

In [51]:
cv_gbt_model.avgMetrics

Out[66]: [0.8633184252847457,
 0.8633184252847457,
 0.8633184252847457,
 0.8633184252847457,
 0.8633184252847457,
 0.8703695893815853,
 0.8703695893815853,
 0.8703695893815853,
 0.8703695893815853,
 0.8703695893815853,
 0.8783545444129885,
 0.8783545444129885,
 0.8783545444129885,
 0.8783545444129885,
 0.8783545444129885,
 0.7733662923002798,
 0.7733662923002796,
 0.7733662923002798,
 0.7733662923002798,
 0.7733662923002798,
 0.7733662923002798,
 0.7733662923002798,
 0.7733662923002798,
 0.7733662923002798,
 0.7733662923002798,
 0.7733662923002796,
 0.7733662923002798,
 0.7733662923002796,
 0.7733662923002798,
 0.7733662923002798,
 0.7657333932614926,
 0.7657333932614926,
 0.7657333932614926,
 0.7657333932614926,
 0.7657333932614926,
 0.7614759892049279,
 0.7614759892049279,
 0.7614759892049279,
 0.7614759892049279,
 0.7614759892049279,
 0.7614759892049279,
 0.7614759892049279,
 0.7614759892049279,
 0.7614759892049279,
 0.7614759892049279,
 0.8639792398741235,
 0.8639792398741235,
 0.8639792398741235,
 0.8639792398741235,
 0.8639792398741235,
 0.8776569992792966,
 0.8776569992792966,
 0.8776569992792966,
 0.8776569992792966,
 0.8776569992792966,
 0.8849526633636242,
 0.884952663363624,
 0.884952663363624,
 0.884952663363624,
 0.884952663363624,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7657338537669935,
 0.7657338537669935,
 0.7657338537669935,
 0.7657338537669935,
 0.7657338537669935,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.8753456964845252,
 0.8753456964845252,
 0.8753456964845252,
 0.8753456964845252,
 0.8753456964845252,
 0.8825082186398436,
 0.8825082186398439,
 0.8825082186398436,
 0.8825082186398434,
 0.8825082186398436,
 0.8897826543415988,
 0.8897826543415985,
 0.889782654341599,
 0.8897826543415989,
 0.8897826543415988,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7733849986477823,
 0.7657338537669935,
 0.7657338537669935,
 0.7657338537669935,
 0.7657338537669935,
 0.7657338537669935,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176,
 0.7614807600419176]

In [52]:
from pyspark.mllib.evaluation import MulticlassMetrics
scoreandlabels = gbresult.rdd.map(lambda z: (z["label"], z["prediction"]))
metric = MulticlassMetrics(scoreandlabels)


In [53]:
#model_stages = cv + cv_svm + cv_nb+cv_rf +cv_gbt
#pipelined_models = Pipeline(stages= model_stages).fit(trainDF)
#pipelined_result = pipelined_models.transform(testDF)

#MODEL ACCURACY 
(on training data)

In [55]:
print('Models and their Performance',"\n")
print('Logistic Regression',evaluator.evaluate(pred_training_cv, {evaluator.metricName: "areaUnderROC"}))
print('Support Vector Machine',svmevaluator.evaluate(pred_training_svm, {svmevaluator.metricName: "areaUnderROC"}))
print('Naive Bayes', nbevaluator.evaluate(pred_training_nb, {nbevaluator.metricName: "areaUnderROC"}))
print('Random forest', rfevaluator.evaluate(pred_training_rf, {rfevaluator.metricName: "areaUnderROC"}))
print('Gradient Boost', gbtevaluator.evaluate(pred_training_gbt, {gbtevaluator.metricName: "areaUnderROC"}))


Models and their Performance 

Logistic Regression 0.9018785721227884
Support Vector Machine 0.8967160185508454
Naive Bayes 0.7971001963021456
Random forest 0.875723801543902
Gradient Boost 0.8921736262323415

#MODEL PREDICTION ACCURACY
(on test data)

In [57]:
print('Models and their Performance',"\n")
print('Logistic Regression',evaluator.evaluate(pred_test_cv, {evaluator.metricName: "areaUnderROC"}))
print('Support Vector Machine',svmevaluator.evaluate(pred_test_svm, {svmevaluator.metricName: "areaUnderROC"}))
print('Naive Bayes', nbevaluator.evaluate(pred_test_nb, {nbevaluator.metricName: "areaUnderROC"}))
print('Random forest', rfevaluator.evaluate(pred_test_rf, {rfevaluator.metricName: "areaUnderROC"}))
print('Gradient Boost', gbtevaluator.evaluate(pred_test_gbt, {gbtevaluator.metricName: "areaUnderROC"}))

Models and their Performance 

Logistic Regression 0.9060457277769502
Support Vector Machine 0.9002626996588036
Naive Bayes 0.8008644664305182
Random forest 0.8784316766552883
Gradient Boost 0.8934300783341731

#ROC V/S PR

In [59]:
print('Models and their Performance',"\n")
print('Logistic Regression: ROC: ',evaluator.evaluate(pred_training_cv, {evaluator.metricName: "areaUnderROC"}), ', PR: ',evaluator.evaluate(pred_training_cv, {evaluator.metricName: "areaUnderPR"}))
print('Support Vector Machine',svmevaluator.evaluate(pred_training_svm, {svmevaluator.metricName: "areaUnderROC"}), ', PR: ',svmevaluator.evaluate(pred_training_svm, {svmevaluator.metricName: "areaUnderPR"}))
print('Naive Bayes', nbevaluator.evaluate(pred_training_nb, {nbevaluator.metricName: "areaUnderROC"}),', PR: ' , nbevaluator.evaluate(pred_training_nb, {nbevaluator.metricName: "areaUnderPR"}))
print('Random forest', rfevaluator.evaluate(pred_training_rf, {rfevaluator.metricName: "areaUnderROC"}), ', PR: ', rfevaluator.evaluate(pred_training_rf, {rfevaluator.metricName: "areaUnderPR"}))
print('Gradient Boost', gbtevaluator.evaluate(pred_training_gbt, {gbtevaluator.metricName: "areaUnderROC"}),', PR: ' , gbtevaluator.evaluate(pred_training_gbt ,{gbtevaluator.metricName: "areaUnderPR"}))

Models and their Performance 

Logistic Regression: ROC: 0.901878572122784 , PR: 0.7681686474339882
Support Vector Machine 0.8967160185508478 , PR: 0.7597095009407047
Naive Bayes 0.7971001963021511 , PR: 0.53013090623475
Random forest 0.8757238015439036 , PR: 0.7187400976812648
Gradient Boost 0.8921736262323415 , PR: 0.749006745002413